In [281]:
import numpy as np
import pandas as pd
import math
import operator

In [282]:
def norma(a):
  a = np.array(a)
  at = a.T
  return math.sqrt(np.dot(a,at)) 

In [283]:
def normalizar(a):
  a = np.array(a)
  return a/norma(a)

In [284]:
def dist(a,b):
  a = np.array(a)
  b = np.array(b)
  return norma(b-a)

In [285]:
def projecao(a,b):
  a = np.array(a)
  b = np.array(b)
  return np.dot(a,b)/np.dot(a,a)*a

In [286]:
def cos(a,b):
    p = projecao(a,b)
    return norma(p)/norma(b)

In [287]:
def avarage_vector(a):
    a = np.array(a)
    return np.mean(a,axis=0)

In [288]:
a = np.array([[1,2,3],[2,3,4],[7,8,9]])
avarage_vector(a)

array([3.33333333, 4.33333333, 5.33333333])

In [289]:
def tratar_df(df): # trata o dataframe, excluindo colunas inuteis.
    df = df.loc[:, 'pace':'goalkeeping_reflexes']
    df = df.fillna(0)
    df = df.fillna(' ')
    df = df.to_numpy()
    return df

In [290]:
def gerar_dfs(df, pos):
    dfs = []
    for i in pos: 
        dfs.append(avarage_vector(tratar_df(df[df['club_position'] == i])))
    return dfs

In [291]:
def predict(dict, pos_nomes, player):
    coss = []
    for p in pos_nomes:
        coss.append(cos(player[0], dict[p]))
    pos = {pos_nomes[i]:coss[i] for i in range(len(pos_nomes))}
    return max(pos, key=pos.get)


In [292]:
def team_over(df, team, ata, mei, deff):
    over = [0,0,0,0]
    over_ = [0,0,0]
    df = df[df['club_name']==team]
    df = df[df['club_position'] != 'RES']
    for _,player in df.iterrows():
        if any(item in player['player_positions'] for item in ata):
            over[0] = over[0] + player['overall']
            over_[0] = over_[0] + 1
        if any(item in player['player_positions'] for item in mei):
            over[1] = over[1] + player['overall']
            over_[1] = over_[1] + 1
        if any(item in player['player_positions'] for item in deff):
            over[2] = over[2] + player['overall']
            over_[2] = over_[2] + 1
    over[0] = over[0]/over_[0]
    over[1] = over[1]/over_[1]
    over[2] = over[2]/over_[2]
    over[3] = (over[0]+over[1]+over[2])/3
    return over
    


In [362]:
from ast import operator

def recommend(df, player):
    player_df = df[df['short_name'] == player]
    d = player_df['player_positions'].to_list()[0]
    d[1]
    print(d)
    for i in player_df['player_positions']:
        print(i)
        print(player_dist(df, "L. Messi", i))
    

def player_dist(df, player, pos):
    player_df = df[df['short_name'] == player]
    dfs = df.drop(df[df['short_name'] == player].index)	# remove o player do df
    dfs = dfs[dfs['player_positions'].str.contains(pos)]	# os players que possuem a mesma posição
    dfs_ = tratar_df(dfs)
    player_df = tratar_df(player_df)
    dists = []
    for i in dfs_:
        dists.append(cos(i, player_df[0])/dist(i, player_df[0]))
    dists = dict(zip(dfs['short_name'], dists))  
    dists = dict(sorted(dists.items(), key = lambda x: x[1], reverse = True))
    return dists

In [326]:
ata_pos = ["ST", "LF", "CF", "RF", "LW", "RW"] # posicoes do atacante
mei_pos = ["CAM", "CM", "CDM", "LM", "RM"] # posicoes do meio
def_pos = ["CB", "LB", "RB", "LWB", "RWB", "GK"] # posicoes do defesa
pos_nomes = ata_pos + mei_pos + def_pos
df = pd.read_csv('players_22.csv')
pos_df = gerar_dfs(df, pos_nomes)
pos_dict = dict(zip(pos_nomes, pos_df))

C:\Users\bieli\AppData\Local\Temp/ipykernel_27856/1556863144.py:5: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('players_22.csv')


In [295]:
team_over(df, 'Tottenham Hotspur', ata_pos, mei_pos, def_pos)

[79.42857142857143, 79.83333333333333, 79.54545454545455, 79.60245310245311]

In [308]:
pl_teams = df[df['league_name'] == 'English Premier League']
pl_over = []
for team in pl_teams['club_name'].unique():
    pl_over.append(team_over(df, team, ata_pos, mei_pos, def_pos))
pl_over = np.array(pl_over).mean()
print(pl_over)
    

76.71144253894253


In [354]:
player_dist(df, "L. Messi", "ST")

{'W. Ben Yedder': 0.01645761128446987,
 'K. Mbappé': 0.01639897139135802,
 'Cristiano Ronaldo': 0.01592661342108092,
 'Iago Aspas': 0.015634704356353935,
 'J. Iličić': 0.014685851730814771,
 'N. Fekir': 0.01468493068934178,
 'S. Agüero': 0.01467998350856991,
 'K. Benzema': 0.014123137724377637,
 'P. Aubameyang': 0.014099891332220007,
 'João Félix': 0.013951424204372391,
 'L. Muriel': 0.013857099147615152,
 'A. Griezmann': 0.01376050832494287,
 'Ferran Torres': 0.01367065125584214,
 'A. Plea': 0.013451569373509699,
 'D. Mertens': 0.013428911685858273,
 'Anderson Talisca': 0.013351260314502999,
 'Y. Carrasco': 0.013242577409719193,
 'A. Sánchez': 0.01318270697853427,
 'A. Kramarić': 0.013132824584670579,
 'M. Rashford': 0.013080066030708154,
 'Gonçalo Guedes': 0.013002351157629991,
 'R. Lewandowski': 0.012948513809212117,
 'E. Zahavi': 0.012891496507047146,
 'Lucas Pérez': 0.012650617972466575,
 'Gerard Moreno': 0.012590561502952423,
 'Morales': 0.012569315817257794,
 'A. Martial': 0.012

In [363]:
recommend(df, "L. Messi")

['RW, ST, CF']
RW, ST, CF
{'B. Mbeumo': 0.009595560267703834, 'G. Mainero': 0.006510923377151956, 'I. Stoica': 0.005489637807436856}
